# Jupyter Environment and Markdown tricks
Functions in `jupyter_utils.py`
***

In [1]:
import IPython
IPython.__version__
# '7.11.0'

from IPython import get_ipython
from IPython.display import Markdown

---
### The html `<figure>` tag usage example I give in [README](./README.md) uses a picture from a url, this one uses a local file:

**Code in next Markdown cell:**
```
<div class="alert alert-info"><p style="font-size:1.2em"><b>Info: </b><br>&emsp;&emsp;* img.src can be local<br>&emsp;&emsp;* img.title attribute holds the tooltip text</p></div>

<figure style="display:inline-block; text-align:center">
  <img src="images/Finley.JPG" 
       alt="x"
       style="display:block; width:400px; height:400px;"
       title="John Finley Walk">
  <figcaption style="color:teal; font-weight:bold; font-family: Arial, Helvetica, sans-serif;">
             Figure 2 (local) - <a href="https://en.wikipedia.org/wiki/John_Huston_Finley">John Finley</a>
  </figcaption>
</figure>
```

**Output:**

<div class="alert alert-info"><p style="font-size:1.2em"><b>Info: </b><br>&emsp;&emsp;* img.src can be local<br>&emsp;&emsp;* img.title attribute holds the tooltip text</p></div>

<figure style="display:inline-block; text-align:center">
  <img src="images/Finley.JPG" 
       alt="x"
       style="display:block; width:400px; height:400px;"
       title="John Finley Walk">
  <figcaption style="color:teal; font-weight:bold; font-family: Arial, Helvetica, sans-serif;">
             Figure 2 (local) - <a href="https://en.wikipedia.org/wiki/John_Huston_Finley">John Finley</a>
  </figcaption>
</figure>

---
### Function to detect whether a notebook is running as a stand-alone Jupyter notebook, or as a JupyterLab notebook. 

In [2]:
def is_lab_notebook():
    import re
    import psutil

    return any(re.search('jupyter-lab-script', x)
               for x in psutil.Process().parent().cmdline())


print('is_lab_notebook() =', is_lab_notebook())

is_lab_notebook() = True


---
### Functions to insert a new cell, with option to overite the current code cell with its output.  
Note: See <i><b>Section 1</b></i> for details.

In [3]:
def ipy_set_next_input(cell_contents, replace_cell):
    if replace_cell:
        shell = get_ipython()

        payload = dict(
            source='set_next_input',
            text=cell_contents,
            replace=True,
        )
        shell.payload_manager.write_payload(payload, single=False)
    else:
        return Markdown(cell_contents)
    

def insert_md_div(div_text_format, div_data=None, replace_cell=False):
    """
    Output Markdown of string with inserted data.
    div_text_format, e.g.: 
               "this is my data: {}" if div_data=x or [x]
               "that too: {}, {}" if div_data=[x,y] or (x,y)
     => assume # place holders == # data items.
    """
    if div_data is None or '{' not in div_text_format:
        # todo: flag error? (bc function not needed)
        return Markdown(div_text_format)

    mismatch_err = 'Format placeholders != data items'
    if isinstance(div_data, (tuple, list)):
        if div_text_format.count('{}') != len(div_data):
            raise IndexError(mismatch_err)
        div = div_text_format.format(*div_data)
        
    elif isinstance(div_data, dict):
        if '{}' not in div_text_format:
            # eg, assume keys given as indices: 'a is {a}, b is {b}'
            div = div_text_format.format_map(div_data)
        else:
            if div_text_format.count('{}') != len(div_data.values()):
                raise IndexError(mismatch_err)
            div = div_text_format.format(*div_data.values())

    else:
        if div_text_format.count('{}') != 1:
            raise IndexError(mismatch_err)
        div = div_text_format.format(div_data)

    return ipy_set_next_input(div, replace_cell)


def insert_alert_div(div_class, div_header=None, div_text=None, 
                     use_class_as_header=True,
                     replace_cell=False):
    """
    use_class_as_header: if div_header=None, div_class is used if 
                         use_class_as_header=True
    Behaviour with `replace_cell=True`:
    The cell is overwritten with the output string:
    ```
    [x]
    alert_div('warning', 'Tip: ', 'some tip here', replace_cell=True)
    The [x] cell becomes:
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    However, the cell mode is still in 'code' not 'markdown', so the desired output
    is obtained only after *manually* changing the cell mode to Markdown.
    
    If `replace_cell=False (default)`, the Markdown string is displayed as any code output cell,
    i.e. below the code cell.
    """
    accepted = ['info', 'warning', 'danger']
    if div_class.lower() not in accepted:
        s = f'<div class="alert"><b>Wrong class:</b> `div_class` not in {accepted}</div>'
        return Markdown(s)
    
    div_class = div_class.lower()
    div_class = 'alert-' + div_class
    div = f'<div class="alert {div_class}"><p style="font-size:1.2em">'
    
    headr = div_header is not None
    if use_class_as_header and not headr:
        div_header = div_class.capitalize()
        headr = True
        
    if headr:
        div_header = div_header.capitalize()
        div += f'<b>{div_header}</b>'
        
    if div_text is not None:
        div_text =  div_text[0].upper() + div_text[1:]
        if headr:
            div += f'<br>&emsp;&emsp;{div_text}'
        else:
            div += f'&ensp;{div_text}'
    
    div += '</p></div>'

    return ipy_set_next_input(div, replace_cell)

<div style="text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"><h3>Section 1</h3>Workaround for using variables in a Markdown cell of a JupyterLab notebook</div>

# The need: 

Let variable v be 123.  
In a Markdown cell of a classic notebook, this code, `My variable is: {{v}}.` will be rendered as "My variable is: 123.", which is expected.  
However, and for reasons I still don't understand, in a lab notebook, the same line will be rendered as "My variable is: {{v}}.", i.e. the smart rendering is lost.

#### This is a documented "Markdown in JupyterLab" [limitation](https://github.com/ipython-contrib/jupyter_contrib_nbextensions/issues/1360) issue (as of Jan, 2020).

## Actual demo (will not fail in a classic notebook):

In [4]:
v = 123

### The next line in this *Markdown cell* is `My variable is: {{v}}`:  <br>

My variable is: {{v}}

## A workaround is to use the Markdown function from IPython.display module:

In [5]:
Markdown(f'My variable is: {v}.')

My variable is: 123.

### I find the use of `display.Markdown` too limiting, so I wrote some functions.

---
# Functions:


### 1. `insert_md_div` is used to compose markdown cells that use variables in jupyterlab. 
### 2. `insert_alert_div` is used to style markdown cells as visual blocks. 

(These functions use `ipy_set_next_input` defined in the first code cell.)

## Caveats / To Do's:

The behavior of these functions depends on the value of argument `replace_cell (bool, default False)`:
* `False` leads to the usual: the code cell contains the function call and a Markdown output cell is created.
* `True` will replace the code cell with the Markdown string

Unfortunately, I have not yet found out how to programmatically:  
a) change the format of the replaced code cell to Markdown  
b) run it  

Operations a and b have to be done mannually.

---
### Variables to format section headers for use with `ipy_set_next_input`:
**Call example:** `ipy_set_next_input(section1, True)`

In [6]:
HDR_style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"

title1='<h3>Section 1</h3>Workaround for using variables in a Markdown cell of a JupyterLab notebook'
section1 = f'<div style="{HDR_style}">{title1}</div>'

title2 = '<h3>Section 2</h3>Format selected string portions with bold'
section2 = f'<div style="{HDR_style}">{title2}</div>'

title3 = '<h3>Section 3</h3>Displaying images with a caption'
section3 = f'<div style="{HDR_style}">{title3}</div>'

title4 = '<h3>Section 4</h3>Miscellany'
section4 = f'<div style="{HDR_style}">{title4}</div>'

***
***
# A. Call examples for `insert_md_div`(div_text_format, div_data=None, replace_cell=False)

### Test data for `insert_md_div`:

In [7]:
# A value
x1 = 20
# A list
x2 = [20, 4]
# A dict
x3 = {'a':20, 'b':4, 'ooo':0}

# A format string preset with dict keys
str_with_keys_as_indices = 'k1 is: {a}, k2 is: {b}, k3 is: {ooo}'

###  A.1: Various calls of `insert_md_div` with output in new cell (default):

In [8]:
insert_md_div('$Variable: {}$', x1)

$Variable: 20$

In [9]:
insert_md_div('Variables: {}, {}', x2)

Variables: 20, 4

In [10]:
insert_md_div(str_with_keys_as_indices, x3)

k1 is: 20, k2 is: 4, k3 is: 0

### A.2:  Call of `insert_md_div` in same cell by passing `replace_cell=True`:
```
insert_md_div("**variable**: {}", x1, replace_cell=True)  // input code
```
```
**variable**: 20  // (same cell) output
```

* <strong>Note: The Code cell must be changed to Markdown and executed: I have not found a way to do this programmatically (yet)!

In [ ]:
**variable**: 20

***
***
# B. Call examples for `insert_alert_div`
```
insert_alert_div(div_class,
                 div_header=None, div_text=None, 
                 use_class_as_header=True,  # applied if div_header=None
                 replace_cell=False)
```

### B.1: `insert_alert_div` in new cell with various alerts:

A blank div with the info-class color:

In [12]:
insert_alert_div('info', use_class_as_header=False)

<div class="alert alert-info"><p style="font-size:1.2em"></p></div>

In [13]:
insert_alert_div('info')

<div class="alert alert-info"><p style="font-size:1.2em"><b>Alert-info</b></p></div>

The switch `use_class_as_header=True` does not apply if div_header is provided:

In [14]:
insert_alert_div('info', 'Information:', 'some info here', use_class_as_header=True)

<div class="alert alert-info"><p style="font-size:1.2em"><b>Information:</b><br>&emsp;&emsp;Some info here</p></div>

In [15]:
insert_alert_div('info', 'info header without text')

<div class="alert alert-info"><p style="font-size:1.2em"><b>Info header without text</b></p></div>

In [16]:
insert_alert_div('info', div_text='info line without header', use_class_as_header=False)

<div class="alert alert-info"><p style="font-size:1.2em">&ensp;Info line without header</p></div>

In [17]:
insert_alert_div('warning', div_text='some warning HERE.')

<div class="alert alert-warning"><p style="font-size:1.2em"><b>Alert-warning</b><br>&emsp;&emsp;Some warning HERE.</p></div>

In [18]:
insert_alert_div('danger', 'Wait!: ', 'Use with caution!!')

<div class="alert alert-danger"><p style="font-size:1.2em"><b>Wait!: </b><br>&emsp;&emsp;Use with caution!!</p></div>

In [19]:
insert_alert_div('comment', 'Wait!: ', 'Use with caution!!')

<div class="alert"><b>Wrong class:</b> `div_class` not in ['info', 'warning', 'danger']</div>

### B.2:  `insert_alert_div` in same cell (with same caveats as in A: change to Markdown, then execute needed): 
Before execution, the next cell contained:
```
insert_alert_div('info', 'info: ', 'some info here', replace_cell=True)
```

In [ ]:
<div class="alert alert-info"><p style="font-size:1.2em"><b>Info: </b><br>&emsp;&emsp;Some info here</p></div>

***
# C. A more elaborate Latex in Markdown example:

<div class="alert alert-info"><p style="font-size:1.2em"><b>Note:</b><br>&emsp;&emsp;A markdown cell should display latex (i.e. in many cases, there is no need for <i>display.Latex</i>)</p></div>

In [21]:
import numpy as np

def poly_to_latex_str(p):
    terms = ['{:.2g}'.format(p.coef[0])]
    if len(p) > 1:
        term = 'x'
        c = p.coef[1]
        if c != 1:
            term = ('{:.2g}'.format(c)) + term
        terms.append(term)
    if len(p) > 2:
        for i in range(2, len(p)):
            term = 'x^{:d}'.format(i)
            c = p.coef[i]
            if c != 1:
                term = ('{:.2g}'.format(c)) + term
            terms.append(term)
    px = '$P(x) = {:s}$'.format(' + '.join(terms))
    px += ', $x \in [{:.2g},\ {:.2g}]$'.format(*p.domain)
    return px

In [22]:
p = np.polynomial.Polynomial([1,2,3], domain=[-10, 10])
print('Polynomial repr from numpy:\n\t{!r}'.format(p))

insert_md_div('**New format:**<br>&emsp;&emsp;' + poly_to_latex_str(p))

Polynomial repr from numpy:
	Polynomial([1., 2., 3.], domain=[-10.,  10.], window=[-1,  1])


**New format:**<br>&emsp;&emsp;$P(x) = 1 + 2x + 3x^2$, $x \in [-10,\ 10]$

#### Same with cell replacement:

In [23]:
# Prep inputs:

px = '**New format:**<br>&emsp;&emsp;$P(x) = {} + {}x + {}x^2$, $x \\in [{},\\ {}]$'
data = [1,2,3,-5,5]

**Call**: `insert_md_div(px, data, replace_cell=True)`

In [ ]:
**New format:**<br>&emsp;&emsp;$P(x) = 1 + 2x + 3x^2$, $x \in [-5,\ 5]$

<div style="text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"><h3>Section 2</h3>Format selected string portions with bold</div>

# Function: format_with_bold

In [24]:
def format_with_bold(s_format, data=None):
    """
    Returns the string with all placeholders preceeded by '_b' replaced 
    with a bold indicator value;
    
    :param: s_format: a string format; 
            if contains '_b{}b_' this term gets bolded.
    :param: s: a string or value
    
    Note 1: '... _b{}; something {}b_ ...' is a valid format.
    Note 2: IndexError is raised using the output format only when
            the input tuple length < number of placeholders ({});
            it is silent when the later are greater (see Example).
    
    Example:
    # No error:
    fmt = 'What! _b{}b_; yes: _b{}b_; no: {}.'
    print(format_with_bold(fmt).format('Cat', 'dog', 3, '@no000'))
    # IndexError:
    print(format_with_bold(fmt).format('Cat', 'dog'))
    """
    if data is None:
        raise TypeError('Missing data (is None).')
    if '{' not in s_format:
        raise TypeError('Missing format placeholders.')
                    
    # Check for paired markers:
    if s_format.count('_b') != s_format.count('b_'):
        err_msg1 = "Bold indicators not paired. Expected '_b with b_'."
        raise LookupError(err_msg1)
    
    # Check for start bold marker:
    b1 = '_b'
    i = s_format.find(b1 + '{')
    
    # Check marker order: '_b' before 'b_':
    if i > s_format.find('}' + 'b_'):
        err_msg2 = "Starting bold indicator not found. Expected '_b before b_'."
        raise LookupError(err_msg2)
        
    while i != -1:
        # Check for trailing bold marker:
        b2 = 'b_'
        j = s_format.find('}' + b2)
        
        if j != -1:
            s_format = s_format.replace(b1, '\033[1m')
            s_format = s_format.replace(b2, '\033[0m')
        else:
            err_msg3 = "Trailing bold indicator not found. Expected '_b with b_'."
            raise LookupError(err_msg3)
            
        i = s_format.find(b1 + '{')
    
    # Now combine string with data:
    mismatch_err = 'Format placeholders != data items'
    
    if isinstance(data, (tuple, list)):
        if s_format.count('{}') != len(data):
            raise IndexError(mismatch_err)
        return s_format.format(*data)
    
    elif isinstance(data, dict):
        if '{}' not in s_format:
            # eg, assume keys given as indices: 'a is {a}, b is {b}'
            return s_format.format_map(data)
        else:
            if s_format.count('{}') != len(data.values()):
                raise IndexError(mismatch_err)
            return s_format.format(*data.values())
    else:
        if s_format.count('{}') != 1:
            raise IndexError(mismatch_err)
        return s_format.format(data)

In [25]:
# Well formated input:

fmt1 = 'Sometimes _b{}b_ {} _b{}b_.'
data = ['Cat', 'is', 'away']
print(format_with_bold(fmt1, data))

Sometimes Cat is away.


In [26]:
# As long as the bold indicators are balanced & ordered, they can be anywhere:

fmt2 = 'Often, but not on Tuesdays, _b{} {} far {}b_.'
print(format_with_bold(fmt2, data))

Often, but not on Tuesdays, Cat is far away.


***
***

<div style="text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"><h3>Miscellany</h3></div>

---
# UTF-8 Geometric Shapes 
[(List)](https://www.w3schools.com/charsets/ref_utf_geometric.asp)

### Ranges:
* Decimal: 9632-9727
* Hex: 25A0-25FF

### Usage:
```
<p>A tiny triangle: &#x25FA;</p>
```
Output:
<p>A tiny triangle:  &#x25FA;</p>

---
# Extra: ANSI excape codes 

See wiki: https://en.wikipedia.org/wiki/ANSI_escape_code

# CSI "Control Sequence Introducer" commands

```
ESC[ 38;5;<n> m Select foreground color
ESC[ 48;5;<n> m Select background color
    0-  7:  standard colors (as in ESC [ 30–37 m)
    8- 15:  high intensity colors (as in ESC [ 90–97 m)
   16-231:  6 × 6 × 6 cube (216 colors): 16 + 36 × r + 6 × g + b (0 ≤ r, g, b ≤ 5)
  232-255:  grayscale from black to white in 24 steps
```

---

### These format should work at the command line as well (...or better!)

In [27]:
s = "Some simple string"
ESC_ = '\033['

print(f"{ESC_}0;0;0m{s}{ESC_}0;0;m")
print(f"{ESC_}0;0;1m{s}{ESC_}0;0;m")
print(f"{ESC_}0;0;4m{s}{ESC_}0;0;m") #4m: underline
#print(f"{ESC_}1;38;41m{s}{ESC_}1;38;0m")
print(f"{ESC_}45;37;4m{s}{ESC_}0;0;m\n")

Some simple string
Some simple string
Some simple string
Some simple string



In [28]:
print(f"{ESC_}0;0;41m{s}{ESC_}0;0;m")
print(f"{ESC_}0;0;42m{s}{ESC_}0;0;m")
print(f"{ESC_}0;0;43m{s}{ESC_}0;0;m")
print(f"{ESC_}0;0;44m{s}{ESC_}0;0;m")

Some simple string
Some simple string
Some simple string
Some simple string


In [29]:
print(f"{ESC_}0;0;45m{s}{ESC_}0;0;m")
print(f"{ESC_}41;4;11m{s}{ESC_}m")
print(f"{ESC_}0;0;46m{s}{ESC_}0;0;m")
print(f"{ESC_}10;0;41m{s}{ESC_}m")

Some simple string
Some simple string
Some simple string
Some simple string


In [30]:
# these may not render properly

print("\033[5;37;0m{}\033[5;37;m".format('Normal text'))
print("\033[5;51;0m{}\033[5;37;m".format('Normal text2'))

print("\033[5;38;1m{}\033[5;38;0m".format('Bold text'))
print("\033[5;38;2m{}\033[5;37;0m".format('Faint text'))

#print("\033[1;31;50m{}\033[0m 1;31;50m \n".format('Some text')) 
#print("\033[1;37;40m{}\033[0m 1;37;40m \n".format('Some text')) 

print("\033[5;37;5m{}\033[5;37;m".format('Bright Colour'))
print("\033[5;37;4m{}\033[5;37;m".format('Underlined text'))

print("\033[3;37;m{}\033[0;37;m".format('Negative Colour'))
print("\033[4;37;m{}\033[0;37;m".format('which? Colour'))
print("\033[5;37;m{}\033[0;37;m".format('Negative Colour'))

Normal text
Normal text2
Bold text
Faint text
Bright Colour
Underlined text
Negative Colour
which? Colour
Negative Colour


***